In [ ]:
!git clone https://github.com/EwaNikodemMasterThesis/AnomalyDetection.git

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
#GroundTruth
TestVideoFile = {}
TestVideoFile[1] = list(range(78,84)) + list(range(95,100)) + list(range(109,111)) + list(range(116,121)) + list(range(392,400)) + list(range(408,410)) + list(range(109,111)) + list(range(503,667)) + list(range(868,876)) + list(range(878,898)) + list(range(900,911)) + list(range(932,1102)) 
TestVideoFile[2] = list(range(273,321)) + list(range(724,765))
TestVideoFile[3] = list(range(295,341)) + list(range(582,623))
TestVideoFile[4] = list(range(380,429)) + list(range(649,693))
TestVideoFile[5] = range(469,787)
TestVideoFile[6] = list(range(345,626)) + list(range(815,826)) + list(range(856,906)) + list(range(921,1008))
TestVideoFile[7] = list(range(300,308)) + list(range(423,495)) + list(range(563,606))
TestVideoFile[8] = range(1,37)
TestVideoFile[9] = list(range(136,184)) + list(range(496,548)) + list(range(551,564)) + list(range(565,567)) + list(range(741,756)) + list(range(875,909)) + list(range(924,938)) + list(range(946,956)) + list(range(969,982)) + list(range(1013,1045)) + list(range(1104,1134)) + list(range(1138,1164))
TestVideoFile[10] = list(range(571,584)) + list(range(593,608)) + list(range(637,657)) + list(range(678,691)) + list(range(699,714)) + list(range(724,756)) + list(range(783,794)) + list(range(805,819))
TestVideoFile[11] = list(range(13,35)) + list(range(48,97)) + list(range(113,126)) + list(range(144,165)) + list(range(308,347))
TestVideoFile[12] = list(range(539,580)) + list(range(592,618)) + list(range(645,659)) + list(range(673,688)) + list(range(695,730)) + list(range(759,796)) + list(range(822,844))
TestVideoFile[13] = list(range(259,287)) + list(range(458,467)) + list(range(473,511))
TestVideoFile[14] = list(range(399,411)) + list(range(417,456)) + list(range(485,508))
TestVideoFile[15] = range(498,588)
TestVideoFile[16] = range(632,741)
TestVideoFile[17] = list(range(1,57)) + list(range(99,427))
TestVideoFile[18] = range(1,295)
TestVideoFile[19] = range(109,249)
TestVideoFile[20] = list(range(65,145)) + list(range(168,242))
TestVideoFile[21] = range(14,67)

In [ ]:
#imports
import os
from os import listdir
from os.path import isfile, join, isdir


from PIL import Image
import numpy as np
import shelve
import keras
import tensorflow as tf 
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, LayerNormalization, MaxPooling2D, UpSampling2D
from tensorflow_addons.layers import MaxUnpooling2D
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

import shutil
import pathlib

import subprocess

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#the paths need to be changed accordingly 
class Config:
  VIDEO_PATH = "/home/user/notebook/UCSD/Avenue_Dataset"
  DATASET_PATH ="/home/user/notebook/UCSD/Avenue_Dataset/training_frames"
  DATASET_PATH_ALLIMAGES ="/home/user/notebook/UCSD/Avenue_Dataset/all_images"
  TEST_PATH ="/home/user/notebook/UCSD/Avenue_Dataset/testing_frames"
  SINGLE_TEST_VIDEO_FILE = 1
  SINGLE_TEST_PATH = "/home/user/notebook/UCSD/Avenue_Dataset/testing_frames/01"
  BATCH_SIZE = 32
  EPOCHS = 50
  MODEL_PATH = "/home/user/notebook/drive/avenue_ae_model_v1.hdf5"
  MODEL_PATH_GEN = "/home/user/notebook/drive/avenue_ae_model_gen_v1.hdf5"
  TRAINING_SET_PATH = "/home/user/notebook/drive/avenue_ae_trainingset_v1.npy"
  THRESHOLD = 0.95

In [ ]:
#!rm -Rf UCSD/Avenue_Dataset/testing_frames
#!rm -Rf UCSD/Avenue_Dataset/training_frames
#!rm -Rf UCSD/Avenue_Dataset/test_frames
#!rm -Rf UCSD/Avenue_Dataset/train_frames

In [ ]:
#dividing videos into frames

def videos_to_frames(root_dataset, out_path):

    for i, (directory, folders, fil) in enumerate(os.walk(root_dataset)):
        for video_name in fil:
            video_path = os.path.join(directory, video_name)

            # creating a folder for each video 
            fname = video_name.split('.avi')[0]
            path_folder = os.path.join(out_path, fname)
            if not os.path.isdir(path_folder):
                os.makedirs(path_folder)

            change_format = 'ffmpeg -i {} -vf fps=25  {}/%04d.jpg'.format(video_path, path_folder)
            subprocess.call(change_format, shell=True)            
  



#training
dataset_root_train = os.path.join(Config.VIDEO_PATH, 'training_videos')

out_split_train = os.path.join(Config.VIDEO_PATH, 'training_frames')
if not os.path.isdir(out_split_train):
    os.mkdir(out_split_train)

#testing
dataset_root_test = os.path.join(Config.VIDEO_PATH, 'testing_videos')

out_split_test = os.path.join(Config.VIDEO_PATH, 'testing_frames')
if not os.path.isdir(out_split_test):
    os.mkdir(out_split_test)



videos_to_frames(dataset_root_train, out_split_train)
videos_to_frames(dataset_root_test, out_split_test)


ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

frame= 1223 fps=498 q=24.8 Lsize=N/A time=00:00:48.92 bitrate=N/A speed=19.9x    
video:10875kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubber

frame= 1017 fps=464 q=24.8 Lsize=N/A time=00:00:40.68 bitrate=N/A speed=18.5x    
video:9257kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

frame= 1511 fps=512 q=24.8 Lsize=N/A time=00:01:00.44 bitrate=N/A speed=20.5x    
video:13714kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubber

frame= 1099 fps=523 q=24.8 Lsize=N/A time=00:00:43.96 bitrate=N/A speed=20.9x    
video:9834kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

...

frame= 1271 fps=527 q=24.8 Lsize=N/A time=00:00:50.84 bitrate=N/A speed=21.1x    
video:11600kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubber

frame=  947 fps=521 q=24.8 Lsize=N/A time=00:00:37.88 bitrate=N/A speed=20.8x    
video:8685kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

frame=  605 fps=494 q=24.8 Lsize=N/A time=00:00:24.20 bitrate=N/A speed=19.8x    
video:5766kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

frame=  426 fps=0.0 q=24.8 Lsize=N/A time=00:00:17.04 bitrate=N/A speed=20.8x    
video:4054kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

frame=  294 fps=0.0 q=24.8 Lsize=N/A time=00:00:11.76 bitrate=N/A speed=20.5x    
video:2826kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberb

In [ ]:
#Copying all frames into 1 catalog to later feed it to generator

pathlib.Path(Config.DATASET_PATH_ALLIMAGES).mkdir(parents=True, exist_ok=True)

for subdir, dirs, files in os.walk(Config.DATASET_PATH):
  for f in files:
    if pathlib.Path(f).suffix == ".jpg":
      shutil.copy(join(Config.DATASET_PATH,subdir,f), join(Config.DATASET_PATH_ALLIMAGES, os.path.basename(subdir).lower()+"_"+f.lower()))

In [ ]:
"""
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/01 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/02 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/03 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/04 | wc -l
!ls /home/user/notebook//UCSD/Avenue_Dataset/training_frames/05 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/06 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/07 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/08 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/09 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/10 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/11 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/12 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/13 | wc -l
!ls /home/user/notebook//UCSD/Avenue_Dataset/training_frames/14 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/15 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/training_frames/16 | wc -l
!ls /home/user/notebook/UCSD/Avenue_Dataset/all_images | wc -l
"""

1364
1511
1487
1511
815
1511
1099
1017
1391
1223
781
145
366
510
353
244
15328


In [ ]:
#pre-processing, reshaping, putting into volumes
def get_clips_by_stride(stride, frames_list, sequence_size):

    clips = []
    size = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 224, 224, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, size, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(np.copy(clip))
                cnt = 0
    return clips


def get_training_set(reload_training_set=True):

    if not reload_training_set:
        return np.load(Config.TRAINING_SET_PATH)
    
    clips = []
    # going over all training folders 
    for f in sorted(listdir(Config.DATASET_PATH)):
        if isdir(join(Config.DATASET_PATH, f)):
            all_frames = []
            # going over all the frames inside that folder
            for c in sorted(listdir(join(Config.DATASET_PATH, f))):
                if str(join(join(Config.DATASET_PATH, f), c))[-3:] == "tif":
                    img = Image.open(join(join(Config.DATASET_PATH, f), c)).convert('L').resize((224, 224))
                    
                    img = np.array(img, dtype=np.float32)
                    img=(img-img.mean())/(img.std())
                    img=np.clip(img,0,1)
                    all_frames.append(img)
            # 10frame-sequences 
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
                
    
    np.save(Config.TRAINING_SET_PATH, clips)
    return clips
    
    

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, frame_list, batch_size):
        self.frame_list = frame_list
        self.batch_size = batch_size

    def __len__(self):
      return (np.ceil(len(self.frame_list) / float(self.batch_size))).astype(np.int)

    def __getitem__(self, index):
      frame_list_temp = self.frame_list[index * self.batch_size : (index+1) * self.batch_size]

      # Set of X_train and y_train
      X, Y = self.__data_generation(frame_list_temp)

      return X, Y

    def __data_generation(self, frame_list_temp):
        frames = []
        for f in frame_list_temp:
          img = Image.open(join(Config.DATASET_PATH_ALLIMAGES, f)).convert('L').resize((224, 224))

          img = np.array(img, dtype=np.float32)
          img=(img-img.mean())/(img.std())
       
          img=np.clip(img,0,1)
          frames.append(img)
        

        augmentation = []
        for stride in range(1, 3):
            augmentation.extend(get_clips_by_stride(stride=stride, frames_list=frames, sequence_size=10))
                
        training_set = np.array(augmentation)
        training_set = training_set.reshape(-1,10,224,224,1)
        
        return training_set, training_set.copy()
        
             


def get_model_gen(reload_model=True):
    if not reload_model:
        return load_model(Config.MODEL_PATH_GEN,custom_objects={'LayerNormalization': LayerNormalization})
    
    
    frames = []
    # loop over the all images
    for f in sorted(listdir(Config.DATASET_PATH_ALLIMAGES)):
      frames.append(f)
            

    training_generator = DataGenerator(frames,Config.BATCH_SIZE)
    
    
    model = Sequential()
    
    #### Encoder ####
    model.add(TimeDistributed(Conv2D(512, (11, 11), strides=4, padding="valid"), batch_input_shape=(None, 10, 224, 224, 1)))  
    model.add(LayerNormalization())
    model.add(TimeDistributed(MaxPooling2D((2, 2), padding="valid")))#, strides = 1)))

    model.add(TimeDistributed(Conv2D(256, (5, 5), strides= 1, padding="same"))) 
    model.add(LayerNormalization())  
    model.add(TimeDistributed(MaxPooling2D((2, 2), padding="valid")))

    model.add(TimeDistributed(Conv2D(128, (3, 3), padding="same"))) 
    

    #### Decoder ####

    model.add(TimeDistributed(Conv2DTranspose(128, (3, 3), padding="same")))  
    model.add(LayerNormalization())
    
    model.add(TimeDistributed(UpSampling2D((2,2))))
    

    model.add(TimeDistributed(Conv2DTranspose(256, (3, 3), padding="valid")))
    model.add(LayerNormalization())
    model.add(TimeDistributed(UpSampling2D((2,2))))

    model.add(TimeDistributed(Conv2DTranspose(512, (5, 5), padding="same")))
    model.add(LayerNormalization())
    model.add(TimeDistributed(Conv2DTranspose(1, (11, 11), strides= 4, padding="same")))
    
    
    print(model.summary())

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=1e-3))
    
    
    model.fit_generator(generator=training_generator,
      epochs = Config.EPOCHS,
      verbose = 1,
      max_queue_size=Config.BATCH_SIZE,
      )
    
    
    model.save(Config.MODEL_PATH_GEN)
    return model
    

In [ ]:
def get_single_test():
    sz = 0

    for subdir, dirs, files in os.walk(Config.SINGLE_TEST_PATH):
      for f in files:
        if pathlib.Path(f).suffix == ".jpg":
          sz = sz + 1

   
    test = np.zeros(shape=(sz, 224, 224, 1))
    
    cnt = 0
    for f in sorted(listdir(Config.SINGLE_TEST_PATH)):
        if str(join(Config.SINGLE_TEST_PATH, f))[-3:] == "jpg":
            img = Image.open(join(Config.SINGLE_TEST_PATH, f)).convert('L').resize((224, 224))
            img = np.array(img, dtype=np.float32)
            img=(img-img.mean())/(img.std())
            img=np.clip(img,0,1)
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test


In [ ]:
#!ls /home/user/notebook//UCSD/Avenue_Dataset/testing_frames/01 | wc -l

In [ ]:
def evaluate(reload_model=False):
    !free -h
    model = get_model_gen(reload_model)
    print("got model")
    test = get_single_test()
    print(test.shape)
    sz = test.shape[0] - 10 + 1
    sequences = np.zeros((sz, 10, 224, 224, 1))

    for i in range(0, sz):
        clip = np.zeros((10, 224, 224, 1))

        for j in range(0, 10):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    print("got data")
    
    
    reconstructed_sequences = []

    index = 0
    batch_size = 100
    while (index * batch_size) < len(sequences): 
      #print('Predict index: ', index)
      sequences_batch = sequences[index * batch_size : (index+1) * batch_size]
      reconstructed_sequences.extend(model.predict(sequences_batch, batch_size=Config.BATCH_SIZE))
      index = index + 1
      tf.keras.backend.clear_session()
    
    #reconstruction error; regularity score
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
    sr = 1.0 - sa

    # plot the regularity scores
    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()

    del model
    del test
    del reconstructed_sequences
    del sa
    
    return sr, sequences


In [ ]:
#!free -h

              razem       użyte       wolne    dzielone   buf/cache    dostępne
Pamięć:        78Gi       5,3Gi        69Gi        82Mi       3,6Gi        72Gi
Wymiana:       2,0Gi          0B       2,0Gi


In [ ]:
pr, before_reconstuction = evaluate(reload_model=False)

In [ ]:
def plotROC(pr):
  y_pred = pr

  sz = 0
  for subdir, dirs, files in os.walk(Config.SINGLE_TEST_PATH):
    for f in files:
      if pathlib.Path(f).suffix == ".jpg":
        sz = sz + 1

  y_test = [1 for element in range(0, sz)]

  for i in TestVideoFile[Config.SINGLE_TEST_VIDEO_FILE]:
    if i < len(y_test):
        y_test[i] = 0

  
  y_test = y_test[5:sz-4]

  if 1 not in y_test: 
    y_test[0] = 1
    

  fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
  fnr = 1 - tpr
  auc = metrics.roc_auc_score(y_test, y_pred)

  eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
  eer = fpr[np.nanargmin(np.absolute((fnr - fpr)))]

  optimal = np.argmax(tpr - fpr)
  optimal_threshold = thresholds[optimal]

  truePositive, falsePositive, falseNegative, trueNegative = 0,0,0,0 
      
  for ii in range(len(pr)):
      if pr[ii]<optimal_threshold and ii in TestVideoFile[Config.SINGLE_TEST_VIDEO_FILE]:
          truePositive +=1
      if pr[ii]<optimal_threshold and ii not in TestVideoFile[Config.SINGLE_TEST_VIDEO_FILE]:
          falsePositive += 1  
      if pr[ii]>optimal_threshold and ii in TestVideoFile[Config.SINGLE_TEST_VIDEO_FILE]:
          falseNegative +=1
      if pr[ii]>optimal_threshold and ii not in TestVideoFile[Config.SINGLE_TEST_VIDEO_FILE]:
          trueNegative += 1 


  #print("FPR: ", fpr)
  #print("TPR: ", tpr)
  #print("THRESHOLDS", thresholds)
  print("AUC: ", auc)
  print("EER: ", eer)
  print("EER THRESHOLD: ", eer_threshold)
  print("Optimal threshold value is:", optimal_threshold)

  print("TP:", truePositive, " FP:", falsePositive, " FN:", falseNegative, "TN: ", trueNegative)

  plt.title('Receiver Operating Characteristic')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')  
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()

  plt.plot(y_test)
  plt.title('Labels')
  plt.ylabel('GT')
  plt.xlabel('Frame')
  plt.show()

  return auc, eer

plotROC(pr)

In [ ]:
clips = []
# going over the training folders (Train000,Train001,..)
for f in sorted(listdir(Config.TEST_PATH)):
    if isdir(join(Config.TEST_PATH, f)):
      if not 'gt' in f:
        clips.append(join(Config.TEST_PATH, f))


scores = []

for i in range(len(clips)):

  Config.SINGLE_TEST_PATH = clips[i]
  Config.SINGLE_TEST_VIDEO_FILE = i+1

  print("PATH: ", Config.SINGLE_TEST_PATH)
  print("GT: ", Config.SINGLE_TEST_VIDEO_FILE)

  pr, before_reconstuction = evaluate()
  scores.append(plotROC(pr))


mean = np.mean(scores, axis=0)
#print(scores)
print("AUC: ", mean[0])
print("EER: ", mean[1])